In [ ]:
#-------------------------- Standard Imports --------------------------#
%reload_ext autoreload
%autoreload 2
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
import acr
import warnings
import pingouin as pg
from scipy.stats import shapiro, normaltest

warnings.filterwarnings('ignore')
from acr.utils import *
probe_ord = ['NNXr', 'NNXo']


from acr.plots import pub, lrg

plt.rcdefaults()
acr.plots.supl()

#--------------------------------- Import Publication Functions ---------------------------------#
pub_utils = acr.utils.import_publication_functions('/home/kdriessen/gh_master/PUBLICATION__ACR/pub_utils.py', 'pub_utils')
from pub_utils import *
data_agg = acr.utils.import_publication_functions('/home/kdriessen/gh_master/PUBLICATION__ACR/data_agg.py', 'data_agg')
from data_agg import *

pu = acr.utils.import_publication_functions('/home/kdriessen/gh_master/PUBLICATION__ACR/pub_utils.py', 'pub_utils')
dag = acr.utils.import_publication_functions('/home/kdriessen/gh_master/PUBLICATION__ACR/data_agg.py', 'data_agg')

In [ ]:
from acr.utils import SOM_BLUE, NNXR_GRAY

MAIN_EXP = 'swisin'
SUBJECT_TYPE = 'som'
MAIN_COLOR = SOM_BLUE

EXP_TYPE_REAL = 'TONIC'

notebook_figure_root = f'{acr.utils.PAPER_FIGURE_ROOT}/swa/tonic_comp'

In [ ]:
subjects, exps = get_subject_list(type=SUBJECT_TYPE, exp=MAIN_EXP)

In [ ]:
full_hyps = {}
hyp_dicts = {}
for subject, exp in zip(subjects, exps):
    full_hyps[subject] = acr.io.load_hypno_full_exp(subject, exp)
    hyp_dicts[subject] = acr.hypnogram_utils.create_acr_hyp_dict(subject, exp, true_stim=True, extra_rebounds=False)

In [ ]:
#load all raw bp dfs
bp_dfs = {}
for sub, exp in zip(subjects, exps):
    bp = load_raw_bp_df(sub, exp)
    bp_dfs[sub] = bp

In [ ]:
# label and make relative to baseline
reldfs = []
for subject in subjects:
    subdf = bp_dfs[subject]
    subdf = acr.hypnogram_utils.label_df_with_hypno_conditions(subdf, hyp_dicts[subject])
    subdf = acr.hypnogram_utils.label_df_with_states(subdf, full_hyps[subject])
    subdf = acr.hypnogram_utils.label_df_with_full_bl(subdf)
    subdf = make_raw_bp_df_relative_to_baseline(subdf, col_to_use='full_bl', value_to_use='True', state_to_use='NREM', method='mean')
    reldfs.append(subdf)

In [ ]:
relbp = pl.concat(reldfs)

In [ ]:
# drop the known bad channels
scd = {}

scd['ACR_44'] = [9, 11, 16]
relbp = drop_sub_channels(relbp, scd)
rebbp = relbp.filter(pl.col('condition') == 'rebound')
reb_avgs = rebbp.group_by(['subject', 'store', 'band']).mean().to_pandas()
reb_avgs = sub_regions_to_df(reb_avgs)

In [ ]:
g = sns.catplot(data=relbp.filter(pl.col('band') == 'delta').filter(pl.col('condition')=='rebound').to_pandas(), 
                x='channel', 
                y='bandpower_rel', 
                hue='store', 
                kind='box',
                row='subject',
                row_order=subjects,
                showfliers=False,
                showmeans=True, meanprops={"marker": "_", 'markerfacecolor':'gold','markeredgecolor':'gold','markersize':'24', "markeredgewidth": "3"},
                height=10,
                aspect=3,
                hue_order=probe_ord, 
                palette=[NNXR_GRAY, MAIN_COLOR]
                )

for ax in g.axes.flat:
    ax.set_ylim(0, 5)

In [ ]:
delta = reb_avgs.query('band == "delta"')
delta = delta.sort_values(by=['subject', 'store'])
rel_delta = delta.query('store == "NNXo"')['bandpower_rel'].values/delta.query('store == "NNXr"')['bandpower_rel'].values

In [ ]:
nnxo = delta.query('store == "NNXo"')['bandpower_rel'].values
nnxr = delta.query('store == "NNXr"')['bandpower_rel'].values

fig_id = 'tonic_delta_rebound_all_chans'
fig_name = f'{SUBJECT_TYPE}__{EXP_TYPE_REAL}__{fig_id}'
fig_path = f'{notebook_figure_root}/{fig_name}.png'

f, ax = acr.plots.gen_paired_boxplot(nnxr, nnxo, colors=[NNXR_GRAY, MAIN_COLOR], alphas=[0.9, 0.6], fsize=(3.5, 4))

ax.set_xlim(0.30, 0.70) # x limits
ax.set_xticklabels(['Contra. Control', 'Optrode'])

plt.savefig(fig_path, dpi=600, bbox_inches='tight', transparent=True)

In [ ]:
# =============================
# ========== STATS ============
# =============================
write = True

diffs = nnxr - nnxo
shap_stat, shap_p = shapiro(diffs) # test the paired differences for normality
print(f'shapiro_p-value: {shap_p}')

stats = pg.ttest(nnxr, nnxo, paired=True)
# stats = pg.wilcoxon(nnxr, nnxo)

hg = pg.compute_effsize(nnxr, nnxo, paired=True, eftype='hedges')
print(f'hedges g: {hg}')

#r = acr.stats.calculate_wilx_r(stats['W-val'][0], len(nnxr))


if write:
    # ==== Write Stats Results ====
    stats_name = f'{fig_name}'
    acr.stats.write_stats_result(stats_name, 'paired_ttest', stats['T'][0], stats['p-val'][0], 'g', hg)
    
    # ===== Write Source Data =====
    source_data = pd.DataFrame({'contra_control': nnxr, 'off_induction': nnxo, 'subject': np.arange(len(nnxr))})
    pu.write_source_data(source_data, stats_name)
stats

In [ ]:
nnxo_rel = nnxo/nnxr
nnxr_rel = nnxr/nnxr

fig_id = 'tonic_delta_rebound_all_chans__NORMALIZED'
fig_name = f'{SUBJECT_TYPE}__{EXP_TYPE_REAL}__{fig_id}'
fig_path = f'{notebook_figure_root}/{fig_name}.png'

f, ax = acr.plots.gen_paired_boxplot(nnxr_rel, nnxo_rel, colors=[NNXR_GRAY, MAIN_COLOR], alphas=[0.9, 0.6], fsize=(3.5, 4), one_sided=True)

ax.set_xlim(0.30, 0.70) # x limits
ax.set_xticklabels(['Contra. Control', 'Optrode'])

plt.savefig(fig_path, dpi=600, bbox_inches='tight', transparent=True)

In [ ]:
nnxo_rel_df = pd.DataFrame({'subject': subjects, 'exp': EXP_TYPE_REAL, 'optrode_rel': nnxo_rel})
pu.write_quick_data(nnxo_rel_df, 'tonic_optrode_delta')

# Full Spectrograms

In [ ]:
reb_spgs = {}
for subject, exp in zip(subjects, exps):
    if subject in scd:
        to_drop = scd[subject]
    else:
        to_drop=None
    reb_spgs[subject] = create_reb_spg(subject, exp, drop_chans=to_drop)
full_df = pd.concat(reb_spgs.values())

In [ ]:
full_means = full_df.groupby(['store', 'frequency']).mean(numeric_only=True).reset_index()

In [ ]:
from statsmodels.stats.multitest import multipletests
# statistical-tests for all frequencies
t_tests = {}
freqs = []
p_vals = []
for freq in full_df['frequency'].unique():
    freqs.append(freq)
    freq_df = full_df.loc[full_df['frequency'] == freq].sort_values(['store', 'subject'])
    nnxr = freq_df.query('store == "NNXr"')['power_rel'].values
    nnxo = freq_df.query('store == "NNXo"')['power_rel'].values
    
    # will use wilcoxon across all frequencies to avoid differing normality assumptions across different frequencies, etc
    stat = pg.wilcoxon(nnxr, nnxo)
    p_vals.append(stat['p-val'][0])

In [ ]:
# Correct for multiple Comparisons using Benjamini–Hochberg False discovery rate correction 
rej, p_fdr, _, _ = multipletests(p_vals, alpha=0.05, method='fdr_bh')

for p_cor, freq in zip(p_fdr, freqs):
    if p_cor<0.05:
        print(round(freq, 1), round(p_cor, 3))
    #print(round(freq, 1), round(p_cor, 3))

In [ ]:
# =============================
# ===== Write Source Data =====
# =============================
dfs = []
for freq in full_df['frequency'].unique():
    freqs.append(freq)
    freq_df = full_df.loc[full_df['frequency'] == freq].sort_values(['store', 'subject'])
    nnxr = freq_df.query('store == "NNXr"')['power_rel'].values
    nnxo = freq_df.query('store == "NNXo"')['power_rel'].values
    f = round(freq, 1)
    fdf = pd.DataFrame({'freq_bin': f, 'contra_control_power': nnxr, 'optrode_power': nnxo, 'subject': np.arange(len(nnxr))})
    dfs.append(fdf)
spg_source_dat = pd.concat(dfs)
pub_utils.write_source_data(spg_source_dat, 'SOM_full_spg_source_data--TONIC')

In [ ]:
fig_id = 'TONIC_full_spg_rebound'
fig_name = f'{SUBJECT_TYPE}__{EXP_TYPE_REAL}__{fig_id}'
fig_path = f'{notebook_figure_root}/{fig_name}.png'

plt.rcdefaults()  # Reset to matplotlib defaults
acr.plots.supl()
plt.rcParams['xtick.bottom'] = True

f, ax = plt.subplots(1, 1, figsize=(4.5, 3))
sns.lineplot(data=full_df, x='frequency', y='power_rel', hue='store', ax=ax, palette=[NNXR_GRAY, MAIN_COLOR], hue_order=probe_ord, errorbar='se', lw=4)

ax.set_xlim(0, 20)  
ax.set_xticks([0, 5, 10, 15, 20])
ax.set_ylabel('')
ax.set_xlabel('Frequency (Hz)')
#remove the legend
ax.legend_.remove()
#ax.axhline(y=1.59, xmin=0.09, xmax=0.15)
#ax.axvspan(0.3, 2.4, ymin=0.985, ymax=1.0)
#ax.axvspan(2.6, 3.5, ymin=0.985, ymax=1.0)

plt.savefig(fig_path, dpi=600, bbox_inches='tight', transparent=True)

In [ ]:
fig_id = 'SPG_1-to-4_Hz'
fig_name = f'SOM-TONIC__{fig_id}'
fig_path = f'{notebook_figure_root}/{fig_name}.png'

plt.rcdefaults()  # Reset to matplotlib defaults
acr.plots.supl()
plt.rcParams['xtick.bottom'] = True

f, ax = plt.subplots(1, 1, figsize=(4.5, 3))
sns.lineplot(data=full_df, x='frequency', y='power_rel', hue='store', ax=ax, palette=[NNXR_GRAY, MAIN_COLOR], hue_order=probe_ord, errorbar='se', lw=4)

ax.set_xlim(0, 20)  
ax.set_xticks([0, 5, 10, 15, 20])
ax.set_ylabel('')
ax.set_xlabel('Frequency (Hz)')
#remove the legend
ax.legend_.remove()
ax.hlines(y=1.58, xmin=1.0, xmax=4.0)

plt.savefig(fig_path, dpi=600, bbox_inches='tight', transparent=True)

In [ ]:
df_delt = full_df.loc[(full_df['frequency'] > 0.7)&(full_df['frequency'] < 4)]
df_delt_means = df_delt.groupby(['subject', 'store']).sum(numeric_only=True).reset_index()
nnxo = df_delt.query('store == "NNXo"')['power_rel'].values
nnxr = df_delt.query('store == "NNXr"')['power_rel'].values

In [ ]:
# =============================
# ========== STATS ============
# =============================
write = True

diffs = nnxr - nnxo
shap_stat, shap_p = shapiro(diffs) # test the paired differences for normality
print(f'shapiro_p-value: {shap_p}')

stats = pg.ttest(nnxr, nnxo, paired=True)
# stats = pg.wilcoxon(nnxr, nnxo)

hg = pg.compute_effsize(nnxr, nnxo, paired=True, eftype='hedges')
print(f'hedges g: {hg}')

#r = acr.stats.calculate_wilx_r(stats['W-val'][0], len(nnxr))


if write:
    # ==== Write Stats Results ====
    stats_name = f'{fig_name}'
    acr.stats.write_stats_result(stats_name, 'paired_ttest', stats['T'][0], stats['p-val'][0], 'g', hg)
    
    # ===== Write Source Data =====
    source_data = pd.DataFrame({'contra_control': nnxr, 'off_induction': nnxo, 'subject': np.arange(len(nnxr))})
    pu.write_source_data(source_data, stats_name)
stats

# NEW SPG

In [ ]:
spgs = {}
for subject, exp in zip(subjects, exps):
    spgs[subject] = data_agg.load_raw_spg(subject, exp)

In [ ]:
def save_quick_spgdf(spg, subject, exp_type):
    path = f'{acr.utils.pub_data_root}/comp_spg-dfs/{subject}--{exp_type}.parquet'
    spg.write_parquet(path)

In [ ]:
for subject in ['ACR_25', 'ACR_39', 'ACR_40', 'ACR_41', 'ACR_44']:
    sspl = dag.df_and_label_spg(spgs[subject], full_hyps, hyp_dicts, subject, flow=0, fhigh=21)
    save_quick_spgdf(sspl, subject, 'tonic')

In [ ]:
fspgs = []
for subject in spgs:
    fspgs.append(data_agg.get_bl_reb_spg_df(spgs[subject], full_hyps, hyp_dicts, subject))

fspg = pl.concat(fspgs)

In [ ]:
def get_relative_rebound_band(fullspg, flow, fhigh):
    band_spg = fullspg.filter(pl.col('frequency') > flow).filter(pl.col('frequency') < fhigh)
    band_spg = band_spg.group_by(['subject', 'datetime', 'store', 'channel', 'state', 'condition']).agg(pl.col('power').sum()).sort('subject','store', 'channel')
    band_spg = data_agg.relativize_df(band_spg, 'condition', 'full_bl', col_to_relativize='power', on=['subject','store', 'channel'])
    rebband = band_spg.cdn('rebound')
    return rebband

In [ ]:
delta_rebound = get_relative_rebound_band(fspg, 0.7, 4)

In [ ]:
bm = delta_rebound.group_by(['subject', 'store', 'channel']).agg(pl.col('power_rel').mean()).sort('subject', 'store', 'channel')
bm = bm.group_by(['subject', 'store']).agg(pl.col('power_rel').mean()).sort('subject', 'store')

In [ ]:
nnxo = bm.prb('NNXo')['power_rel'].to_numpy()
nnxr = bm.prb('NNXr')['power_rel'].to_numpy()

f, ax = acr.plots.gen_paired_boxplot(nnxr, nnxo)

In [ ]:
nnxo_rel = nnxo/nnxr
nnxo_rel_df = pd.DataFrame({'subject': subjects, 'exp': EXP_TYPE_REAL, 'optrode_rel': nnxo_rel})
pu.write_quick_data(nnxo_rel_df, 'tonic_optrode_delta')

In [ ]:
pg.ttest(nnxr, nnxo, paired=True)